In [ ]:
from datasets import load_dataset
import pandas as pd


print("Loading dataset...")
ds = load_dataset("ura-hcmut/PhoMT",split="train", token="__").shuffle(seed=42).select(range(3000))

print(ds)
print(ds.column_names)

for i in range(min(3, len(ds))):
    sample = ds[i]
    print(f"\nSample {i+1}:")
    for key, value in sample.items():
        print(f"  {key}: {value}")
        print("-" * 50)

Loading dataset...


README.md:   0%|          | 0.00/653 [00:00<?, ?B/s]

PhoMT_training.csv:   0%|          | 0.00/525M [00:00<?, ?B/s]

PhoMT_validation.csv: 0.00B [00:00, ?B/s]

PhoMT_test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2977999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/18720 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19151 [00:00<?, ? examples/s]

Dataset({
    features: ['en', 'vi'],
    num_rows: 3000
})
['en', 'vi']

Sample 1:
  en: Daddy, I like your car wash.
--------------------------------------------------
  vi: Bố ơi, con thích tiệm rửa xe của bố.
--------------------------------------------------

Sample 2:
  en: Scarborough sits outside the Spratlys and so is claimed only by China and the Philippines.
--------------------------------------------------
  vi: Bãi Scarborough nằm ngoài quần đảo Trường Sa và do vậy chỉ có Trung Quốc và Philíppin tranh chấp khu vực này.
--------------------------------------------------

Sample 3:
  en: It says here we should work in teams.
--------------------------------------------------
  vi: Đây nói rằng chúng ta nên làm theo nhóm.
--------------------------------------------------


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# Hugging Face model id
model_id = "Helsinki-NLP/opus-mt-en-vi"

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
).to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples["en"],
        max_length=128,
        truncation=True,
        padding=False
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["vi"],
            max_length=128,
            truncation=True,
            padding=False
        )

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"],
    }


print("Preprocessing dataset...")
processed_ds = ds.map(preprocess_function, remove_columns=['en', 'vi'])

print("Processed dataset structure:")
print(processed_ds)

print("\nSample after preprocessing:")
sample = processed_ds[0]
print(f"{sample}")

train_test_split = processed_ds.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"\nTrain samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")


Preprocessing dataset...


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Processed dataset structure:
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3000
})

Sample after preprocessing:
{'input_ids': [3412, 4, 9, 146, 79, 1173, 5395, 2, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [975, 735, 4, 76, 291, 5385, 3288, 493, 21, 517, 2, 0]}

Train samples: 2700
Eval samples: 300


In [4]:
args = Seq2SeqTrainingArguments(
    output_dir="./translation_results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=1,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_steps=10,
    eval_strategy="steps",
    eval_steps=10,
    report_to="none",
)

In [5]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

/tmp/ipython-input-1867562233.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss
10,1.767400,1.891653
20,2.165200,1.859581
30,1.910700,1.864926
40,2.099200,1.858141
50,1.738300,1.842311
60,2.048000,1.832442
70,2.083200,1.822985
80,1.944500,1.825247
90,2.003500,1.817640
100,2.256300,1.823602


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=675, training_loss=1.9709953703703704, metrics={'train_runtime': 481.4989, 'train_samples_per_second': 5.607, 'train_steps_per_second': 1.402, 'total_flos': 20745799335936.0, 'train_loss': 1.9709953703703704, 'epoch': 1.0})

In [6]:
trainer.save_model()
tokenizer.save_pretrained(args.output_dir)

('./translation_results/tokenizer_config.json',
 './translation_results/special_tokens_map.json',
 './translation_results/vocab.json',
 './translation_results/source.spm',
 './translation_results/target.spm',
 './translation_results/added_tokens.json')

In [13]:
!pip install sacrebleu -q
import sacrebleu

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm.auto import tqdm
import random

model_path = "/content/translation_results"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, local_files_only=True, torch_dtype=torch.float16, device_map="auto")
model.eval()

test_ds = load_dataset("ura-hcmut/PhoMT",split="test", token="__").select(range(1000))
print(f"Đã tải tập test: {len(test_ds)} câu")

def translate(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256).to(model.device)
    with torch.no_grad():
        generated = model.generate(
            **inputs,
            max_new_tokens=256,
            num_beams=5,
            early_stopping=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )
    return tokenizer.decode(generated[0], skip_special_tokens=True).strip()

results = []
for item in tqdm(test_ds):
    en = item["en"]
    ref = item["vi"]
    mt = translate(en)
    results.append({"en": en, "ref": ref, "mt": mt})

print("\n" + "="*80)
print("KẾT QUẢ DỊCH TRÊN TẬP TEST")
print("="*80)

random.seed(42)
samples = random.sample(results, 10)

print("\nMẪU DỊCH NGẪU NHIÊN:")
for i, r in enumerate(samples, 1):
    print(f"\n[{i}]")
    print(f"EN : {r['en']}")
    print(f"REF: {r['ref']}")
    print(f"MT : {r['mt']}")
    print("-" * 70)

preds = [r["mt"] for r in results]
refs  = [[r["ref"]] for r in results]

bleu = sacrebleu.corpus_bleu(preds, refs)

print(f"\nĐIỂM SỐ CHÍNH THỨC TRÊN TẬP TEST:")
print(f"SacreBLEU : {bleu.score:.2f}")


Đã tải tập test: 1000 câu


  0%|          | 0/1000 [00:00<?, ?it/s]


KẾT QUẢ DỊCH TRÊN TẬP TEST

MẪU DỊCH NGẪU NHIÊN:

[1]
EN : But I think that Eleen expresses best what we really get out of this, which is the actual joy of collaboration.
REF: Nhưng tôi nghĩ là Eileen thể hiện tốt nhất cái mà chúng tôi thực sự làm được từ dự án này, đó là sự thích thú trong khi cộng tác.
MT : Nhưng tôi nghĩ Eleen diễn đạt tốt nhất những gì chúng ta thực sự nhận được từ điều này, đó là niềm vui thực sự của hợp tác.
----------------------------------------------------------------------

[2]
EN : Elsewhere, a left - foot birthmark means the person will be massively intelligent, while if it's on the right, they'll love exploration, adventure and travel.
REF: Ở những vùng khác, một cái bớt trên chân trái có nghĩa là người đó sẽ cực kỳ thông minh, trong khi nếu nó ở chân phải, họ sẽ thích khám phá, mạo hiểm và du lịch.
MT : Các nhà du lịch, giảm chân trái có nghĩa là con người sẽ thông minh vượt bậc, trong khi nếu ở bên phải, họ sẽ thích khám phá, phiêu lưu và du lịch.
---